In [33]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [35]:
def multiplied_combinations(input):
    # TODO exclude duplicates
    return tf.multiply(
        tf.repeat(input, input.shape[-1], axis=-1),
        tf.tile(
            input,
            tf.concat(
                [
                    tf.ones(len(input.shape) - 1, dtype=tf.int32),
                    tf.constant([input.shape[-1]]),
                ],
                axis=0,
            ),
        ),
    )


assert tf.reduce_all(
    tf.equal(
        multiplied_combinations(tf.constant([[3, 7], [5, 9]])),
        tf.constant([[9, 21, 21, 49], [25, 45, 45, 81]]),
    )
)

In [36]:
def train_model(inputs, outputs):
    input_layer = tf.keras.layers.Input(shape=inputs.shape[1:])
    hidden_layer = input_layer
    for i in range(1):
        hidden_layer = tf.keras.layers.concatenate(
            [
                hidden_layer,
                tf.keras.layers.Lambda(multiplied_combinations)(hidden_layer),
            ]
        )
    output_layer = tf.keras.layers.Dense(outputs.shape[1:][0])(hidden_layer)
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.MeanSquaredError(),
    )
    model.fit(inputs, outputs, epochs=100)
    model.summary()
    return model

In [37]:
def train_model_on_function(input_shape, f):
    inputs = tf.random.uniform(
        shape=(1000,) + input_shape, minval=-10, maxval=10, dtype=tf.float32
    )
    outputs = tf.map_fn(f, inputs)
    model = train_model(inputs, outputs)
    return model

In [38]:
train_model_on_function((2,), f2).predict(tf.constant([[1, 2], [3, 4]]))

Epoch 1/100
32/32 [==============================] - 0s 3ms/step - loss: 1037.1987
Epoch 2/100
32/32 [==============================] - 0s 3ms/step - loss: 904.7590
Epoch 3/100
32/32 [==============================] - 0s 3ms/step - loss: 790.7197
Epoch 4/100
32/32 [==============================] - 0s 4ms/step - loss: 689.4868
Epoch 5/100
32/32 [==============================] - 0s 3ms/step - loss: 600.7325
Epoch 6/100
32/32 [==============================] - 0s 3ms/step - loss: 521.0527
Epoch 7/100
32/32 [==============================] - 0s 3ms/step - loss: 450.0837
Epoch 8/100
32/32 [==============================] - 0s 3ms/step - loss: 389.6075
Epoch 9/100
32/32 [==============================] - 0s 3ms/step - loss: 337.2119
Epoch 10/100
32/32 [==============================] - 0s 3ms/step - loss: 291.8593
Epoch 11/100
32/32 [==============================] - 0s 3ms/step - loss: 252.1346
Epoch 12/100
32/32 [==============================] - 0s 3ms/step - loss: 217.4266
Epoch 13/100

array([[ 2.4003487, -1.0279934],
       [ 6.3251944, -1.0213295]], dtype=float32)

In [39]:
train_model_on_function((2,), lambda i: tf.stack([i[0] + i[1] + 3, i[0] * i[1]])).predict(tf.constant([[2,3], [4,5]]))

Epoch 1/100
32/32 [==============================] - 0s 3ms/step - loss: 1792.8387
Epoch 2/100
32/32 [==============================] - 0s 3ms/step - loss: 1628.4554
Epoch 3/100
32/32 [==============================] - 0s 2ms/step - loss: 1483.3678
Epoch 4/100
32/32 [==============================] - 0s 3ms/step - loss: 1354.1741
Epoch 5/100
32/32 [==============================] - 0s 3ms/step - loss: 1235.6787
Epoch 6/100
32/32 [==============================] - 0s 3ms/step - loss: 1129.3975
Epoch 7/100
32/32 [==============================] - 0s 3ms/step - loss: 1032.0615
Epoch 8/100
32/32 [==============================] - 0s 3ms/step - loss: 942.1371
Epoch 9/100
32/32 [==============================] - 0s 3ms/step - loss: 859.6473
Epoch 10/100
32/32 [==============================] - 0s 3ms/step - loss: 783.2980
Epoch 11/100
32/32 [==============================] - 0s 3ms/step - loss: 712.1791
Epoch 12/100
32/32 [==============================] - 0s 3ms/step - loss: 646.3446
Epoch 

array([[ 6.3379145,  6.1562943],
       [10.955261 , 20.131971 ]], dtype=float32)